In [ ]:
import pysheds
import os
from pysheds.grid import Grid
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray

In [ ]:
ds = xr.open_dataset("../data/srtm1_30m_utah.tif", engine="rasterio")

In [ ]:
grid = Grid.from_raster("../data/srtm1_30m_utah.tif")
dem = grid.read_raster("../data/srtm1_30m_utah.tif")

In [ ]:
type(grid)

In [ ]:
type(dem)

In [ ]:
# Fill depressions
flooded_dem = grid.fill_depressions(dem)

# Resolve flats
inflated_dem = grid.resolve_flats(flooded_dem)
fdir = grid.flowdir(inflated_dem)

In [ ]:
# Specify pour point
x, y = -111.575, 37.778
# Delineate the catchment
catch = grid.catchment(x=x, y=y, fdir=fdir, xytype="coordinate")

# Plot the result
grid.clip_to(catch)
catch_view = grid.view(catch)

In [ ]:
# Plot the catchment
fig, ax = plt.subplots(figsize=(8, 6))
fig.patch.set_alpha(0)

plt.grid("on", zorder=0)
im = ax.imshow(
    np.where(catch_view, catch_view, np.nan),
    extent=grid.extent,
    zorder=1,
    cmap="Greys_r",
)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Delineated Catchment", size=14)

In [ ]:
import fiona

grid = Grid.from_ascii("./data/dir.asc")

# Specify pour point
x, y = -97.294167, 32.73750

# Delineate the catchment
catch = grid.catchment(x=x, y=y, fdir=fdir, xytype="coordinate")

# Clip to catchment
grid.clip_to(catch)

# Create view
catch_view = grid.view(catch, dtype=np.uint8)

# Create a vector representation of the catchment mask
shapes = grid.polygonize(catch_view)

# Specify schema
schema = {"geometry": "Polygon", "properties": {"LABEL": "float:16"}}

# Write shapefile
with fiona.open(
    "catchment.shp",
    "w",
    driver="ESRI Shapefile",
    crs=grid.crs.srs,
    schema=schema,
) as c:
    i = 0
    for shape, value in shapes:
        rec = {}
        rec["geometry"] = shape
        rec["properties"] = {"LABEL": str(value)}
        rec["id"] = str(i)
        c.write(rec)
        i += 1

In [ ]:
shapes = grid.polygonize(catch_view)

In [ ]:
shapes